<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT306 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [24]:
import numpy as np 
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_02.csv'
 ] 
url_codigos = 'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_codigo.csv'



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [25]:
# Lectura
df_anio1 = pd.read_csv(archivos_anio[0])
df_anio2 = pd.read_csv(archivos_anio[1])
df_codigos = pd.read_csv(url_codigos)

# A minúscula
df_anio1.columns = df_anio1.columns.str.lower()
df_anio2.columns = df_anio2.columns.str.lower()
df_codigos.columns = df_codigos.columns.str.lower()

# CONCAT
df_anio = pd.concat([df_anio1, df_anio2], ignore_index=True)

# Merge
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')


In [26]:
df_codigos = df_codigos[(df_codigos['pais'] != 'malo') & (df_codigos['codigo_iso'] != 'NER')]



### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [27]:
df.shape

(3077, 5)

In [28]:
df.columns.tolist()

['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

In [29]:
df.dtypes

codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object

* No hay columnas con un tipo de dato inesperado.

In [30]:
df.describe()

,anio,indice,ranking
count,3077.000000,2680.000000,2854.000000
mean,2009.941176,204.836847,475.715137
std,5.786018,2687.491660,6455.679267
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.490000,35.000000
50%,2009.000000,28.150000,71.000000
75%,2015.000000,41.447500,110.000000
max,2019.000000,64536.000000,121056.000000


Los valores de `indice` y `ranking` presentan valores extremos muy altos, lo que resulta en una desviación estándar muy alta y valores máximos muy alejados de la media y la mediana. Esto sugiere quizá la necesidad de depurar los valores atípicos/outliers.

In [31]:
df.isnull().sum()

codigo_iso      0
anio            0
indice        397
ranking       223
pais            0
dtype: int64

In [32]:
lista = []
for i in range(1, 4):
    proporcion = df.isnull().sum().iloc[i]/df.sum().iloc[i]
    lista.append(round(float(proporcion*100), 3))
lista

[0.0, 0.072, 0.016]

No hay columnas con más del 30% de datos faltantes.

In [33]:
df['pais'].nunique()

180

In [34]:
df['anio'].nunique()

17

In [35]:
int(df.duplicated().sum())

0

In [36]:
coincidencias = df.groupby('codigo_iso')['pais'].nunique()
coincidencias = coincidencias[coincidencias > 1]
print(coincidencias)

codigo_iso
ZWE    2
Name: pais, dtype: int64





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [37]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

df_america['indice'] = pd.to_numeric(df_america['indice'], errors='coerce')
df_america = df_america.dropna(subset=['indice'])

resultados = []

for anio in sorted(df_america['anio'].unique()):
    tmp = df_america[df_america['anio'] == anio]

    i_min = tmp['indice'].idxmin()
    i_max = tmp['indice'].idxmax()

    resultados.append({
        'anio': anio,
        'pais_menor_indice': tmp.loc[i_min, 'pais'],
        'indice_min': tmp.loc[i_min, 'indice'],
        'pais_mayor_indice': tmp.loc[i_max, 'pais'],
        'indice_max': tmp.loc[i_max, 'indice'],
    })

res = pd.DataFrame(resultados)
print(res.head())

   anio  pais_menor_indice  indice_min pais_mayor_indice  indice_max
0  2001             Canadá         0.8              Cuba       90.30
1  2002  Trinidad y Tobago         1.0              Cuba       97.83
2  2003  Trinidad y Tobago         2.0         Argentina    35826.00
3  2004  Trinidad y Tobago         2.0              Cuba       87.00
4  2005            Bolivia         4.5              Cuba       95.00


In [38]:
g = df_america.dropna(subset=['indice']).copy()

idx_min = g.groupby('anio')['indice'].idxmin()
idx_max = g.groupby('anio')['indice'].idxmax()

min_df = (
    g.loc[idx_min, ['anio','pais','indice']]
     .rename(columns={'pais':'pais_menor_indice',
                      'indice':'indice_min'})
)

max_df = (
    g.loc[idx_max, ['anio','pais','indice']]
     .rename(columns={'pais':'pais_mayor_indice',
                      'indice':'indice_max'})
)

res_vec = (
    pd.merge(min_df, max_df, on='anio', how='inner')
      .sort_values('anio', ignore_index=True)
)

print(res_vec.head())


   anio  pais_menor_indice  indice_min pais_mayor_indice  indice_max
0  2001             Canadá         0.8              Cuba       90.30
1  2002  Trinidad y Tobago         1.0              Cuba       97.83
2  2003  Trinidad y Tobago         2.0         Argentina    35826.00
3  2004  Trinidad y Tobago         2.0              Cuba       87.00
4  2005            Bolivia         4.5              Cuba       95.00


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [39]:
tabla = pd.pivot_table(
    df,
    index='pais',
    columns='anio',
    values='indice',
    aggfunc='max',
    fill_value=0
)

print(tabla.head())


anio         2001   2002   2003   2004   2005   2006   2007   2008   2009  \
pais                                                                        
Afghanistán  35.5  40.17  28.25  39.17  44.25  56.50  59.25  54.25  51.67   
Albania       0.0   6.50  11.50  14.17  18.00  25.50  16.00  21.75  21.50   
Alemania      1.5   1.33   2.00   4.00   5.50   5.75   4.50   3.50   4.25   
Algeria      31.0  33.00  43.50  40.33  40.00  40.50  31.33  49.56  47.33   
Andorra       0.0   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

anio          2012   2013   2014   2015   2017   2018   2019  
pais                                                          
Afghanistán  37.36  37.07  37.44  37.75  39.46  37.28  36.55  
Albania      30.88  29.92  28.77  29.92  29.92  29.49  29.84  
Alemania     10.24  10.23  11.47  14.80  14.97  14.39  14.60  
Algeria      36.54  36.26  36.63  41.69  42.83  43.13  45.75  
Andorra       6.82   6.82  19.87  19.87  21.03  22.21  24.63  


In [40]:
pais_max = tabla.max(axis=1).idxmax()
pais_min = tabla.replace(0, np.nan).min(axis=1).idxmin()

print("Mayor indice:", pais_max)
print("Menor indice (≠0):", pais_min)


Mayor indice: Kosovo
Menor indice (≠0): Austria


In [41]:
anio_max = tabla.mean(axis=0).idxmax()
anio_min = tabla.mean(axis=0).idxmin()

print("Año con mayor promedio:", anio_max)
print("Año con menor promedio:", anio_min)


Año con mayor promedio: 2013
Año con menor promedio: 2001


In [42]:
variabilidad = (tabla.max(axis=1) - tabla.min(axis=1))
pais_var = variabilidad.idxmax()

print("País con mayor variabilidad:", pais_var)


País con mayor variabilidad: Kosovo


In [43]:
constantes = tabla[tabla.nunique(axis=1) == 1].index.tolist()
print("Países con índice constante:", constantes)


Países con índice constante: []


In [44]:
sin_datos = tabla[(tabla == 0).all(axis=1)].index.tolist()
print("Países sin datos:", sin_datos)


Países sin datos: []
